In [2]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('./models')))
sys.path.append(os.path.abspath(os.path.join('./Data_Loaders')))
sys.path.append(os.path.abspath(os.path.join('./Loss_Functions')))

# Importing the lightning module and network
import pytorch_lightning as pl
from trainer import CoolSystem

# Importing the models
from arch import SegNet, SegNet_Split
from arch_2 import Depth_Net
from resnet import ResNetUNet, ResNetUNet50
from densenet_2 import FCDenseNet103

# Import the loss functions for evaluation
import loss_functions, smoothness_loss, geometric_loss, photometric_loss

# Importing the data loader
import data_loader
from torch.utils.data import Dataset, DataLoader

# Importing Utils
import argparse
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, utils
import torchvision
import torch
import tqdm
import gc
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

In [3]:
BCELoss          = torch.nn.BCELoss()
Photometric_loss = photometric_loss.Photometric_loss(vgg_model=torchvision.models.vgg16(pretrained=True).eval().cuda())
Smoothness_loss  = smoothness_loss.Smoothness_loss()
Geometric_loss   = geometric_loss.Geometric_loss()

w_photo          = 1.0
w_geometric      = 0.001
w_smooth         = 0.05
w_bce            = 0.3

Alpha value =  0.05


In [5]:
if __name__ == "__main__":
    
    # Path to the output directory
    directory_save = '../delme3-1/'
    if not os.path.exists(directory_save):
        os.makedirs(directory_save)
        
    # Path to the input dataset    
    set_num = 2
    dataset = '/scratch/Warbucks/cars_blender_100/'
    
    # Load the checkpoints
    depth_network = ResNetUNet50()
    depth_network.load_state_dict(torch.load('../../models/04-33-01/ResNetUNet50_bce_0.3_w_photo_1.0_w_smooth_0.05_w_geometric_0.001.ckpt'))
    
    # Load the dataset
    train_data_set = data_loader.MultiView_dataset_blender(dataset, transform = data_loader.MultiView_dataset_blender.ToTensor(), train = set_num, num_views=3, gt_depth=True)
    train_dataloader = DataLoader(train_data_set, batch_size = 1, shuffle=True)
    
    # Move to the GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        depth_network = depth_network.to(device)

    
    depth_network = depth_network.float()
    num_samples = len(train_dataloader.dataset)
        
    # Set the grad tracking to false, for evaluation
    torch.no_grad()
    depth_network.eval()
    
    gt_photometric_loss = []
    gt_geometric_loss = []
    gt_smoothness_loss = []
    gt_mask_loss = []
    gt_total_loss = []
    
    for epoch in range(50):
        depth_network.eval()
        print("%"*20)
        print(f"Epoch: {epoch}")
        print("%"*20)
        for batch, data_sample in enumerate(train_dataloader):

            if torch.cuda.is_available():
                data_sample['depths'] = data_sample['depths'].to(device).float()
                data_sample['intrinsics'] = data_sample['intrinsics'].to(device).float()
                data_sample['masks'] = data_sample['masks'].to(device).float()
                data_sample['views'] = data_sample['views'].to(device).float()
                data_sample['poses'] = data_sample['poses'].to(device).float()

                gt_mask_ref = data_sample['masks'][:, 0]
                gt_depth_ref = data_sample['depths'].to(device).float()
                
                
            
            # Forward Pass
#             output = depth_network(data_sample['views'][:, 0])
#             pr_target_depth = [output[0].to(device).float()]
#             for i in range(1, data_sample['num_views'][0]):
#                 pr_target_depth.append(depth_network(data_sample['views'][:, i])[0].to(device).float())
#             pr_target_depth = torch.cat(pr_target_depth, dim=1)
                    
                
            # Compute Ground Truth Loss
            gt_photometric_loss.append(w_photo       * Photometric_loss(data_sample, gt_depth_ref[:, 0]).data)
            gt_geometric_loss.append(w_geometric   * Geometric_loss(data_sample, gt_depth_ref.squeeze(2)).data)
            gt_smoothness_loss.append(w_smooth      * Smoothness_loss(gt_depth_ref[:, 0], data_sample).data)
            gt_mask_loss.append(w_bce         * BCELoss(gt_mask_ref[:, 0], gt_mask_ref[:, 0]).data)
            gt_total_loss.append(gt_photometric_loss[-1]   + gt_smoothness_loss[-1] + gt_mask_loss[-1] + gt_geometric_loss[-1])
            print("gt_photometric_loss:", gt_photometric_loss[-1])
            print("gt_geometric_loss:", gt_geometric_loss[-1])
            print("gt_smoothness_loss:", gt_smoothness_loss[-1])
            print("gt_mask_loss:", gt_mask_loss[-1])
            print("gt_total_loss:", gt_total_loss[-1])
            print()
            
#             # Compute Prediction Loss
#             pr_photometric_loss = w_photo       * Photometric_loss(data_sample, output[0])
# #             pr_geometric_loss   = w_geometric   * Geometric_loss(data_sample, pr_target_depth)
#             pr_smoothness_loss  = w_smooth      * Smoothness_loss(output[0], data_sample)
#             pr_mask_loss        = w_bce         * BCELoss(output[1], data_sample['masks'][:, 0])
#             pr_total_loss       = pr_photometric_loss + pr_smoothness_loss + pr_mask_loss
# #             pr_total_loss       = pr_photometric_loss + pr_geometric_loss + pr_smoothness_loss + pr_mask_loss
            
            torch.cuda.empty_cache()
            gc.collect()

        GPUtil.showUtilization()

Retrieving data
9800 9800 9800
Data retrieved
%%%%%%%%%%%%%%%%%%%%
Epoch: 0
%%%%%%%%%%%%%%%%%%%%
gt_photometric_loss: tensor(0.3541, device='cuda:0')
gt_geometric_loss: tensor(0.0009, device='cuda:0')
gt_smoothness_loss: tensor(0.0223, device='cuda:0')
gt_mask_loss: tensor(0., device='cuda:0')
gt_total_loss: tensor(0.3773, device='cuda:0')

gt_photometric_loss: tensor(0.4233, device='cuda:0')
gt_geometric_loss: tensor(0.0007, device='cuda:0')
gt_smoothness_loss: tensor(0.0492, device='cuda:0')
gt_mask_loss: tensor(0., device='cuda:0')
gt_total_loss: tensor(0.4732, device='cuda:0')

gt_photometric_loss: tensor(0.5864, device='cuda:0')
gt_geometric_loss: tensor(0.0013, device='cuda:0')
gt_smoothness_loss: tensor(0.0168, device='cuda:0')
gt_mask_loss: tensor(0., device='cuda:0')
gt_total_loss: tensor(0.6045, device='cuda:0')

gt_photometric_loss: tensor(0.3721, device='cuda:0')
gt_geometric_loss: tensor(0.0012, device='cuda:0')
gt_smoothness_loss: tensor(0.0982, device='cuda:0')
gt_mask_l

gt_photometric_loss: tensor(0.0848, device='cuda:0')
gt_geometric_loss: tensor(0.0015, device='cuda:0')
gt_smoothness_loss: tensor(0.0258, device='cuda:0')
gt_mask_loss: tensor(0., device='cuda:0')
gt_total_loss: tensor(0.1121, device='cuda:0')

gt_photometric_loss: tensor(0.5254, device='cuda:0')
gt_geometric_loss: tensor(0.0013, device='cuda:0')
gt_smoothness_loss: tensor(0.0181, device='cuda:0')
gt_mask_loss: tensor(0., device='cuda:0')
gt_total_loss: tensor(0.5448, device='cuda:0')

gt_photometric_loss: tensor(0.1793, device='cuda:0')
gt_geometric_loss: tensor(0.0008, device='cuda:0')
gt_smoothness_loss: tensor(0.0250, device='cuda:0')
gt_mask_loss: tensor(0., device='cuda:0')
gt_total_loss: tensor(0.2052, device='cuda:0')

gt_photometric_loss: tensor(0.2912, device='cuda:0')
gt_geometric_loss: tensor(0.0012, device='cuda:0')
gt_smoothness_loss: tensor(0.0214, device='cuda:0')
gt_mask_loss: tensor(0., device='cuda:0')
gt_total_loss: tensor(0.3138, device='cuda:0')

gt_photometric_l

KeyboardInterrupt: 

In [6]:
a = torch.FloatTensor(gt_total_loss)
print("Train Loss:",torch.mean(a))

a = torch.FloatTensor(gt_photometric_loss)
print("",torch.mean(a))

a = torch.FloatTensor(gt_smoothness_loss)
print(torch.mean(a))

a = torch.FloatTensor(gt_mask_loss)
print(torch.mean(a))

a = torch.FloatTensor(gt_geometric_loss)
print("Geometric ",torch.mean(a))


tensor(0.4145)
tensor(0.3768)
tensor(0.0366)
tensor(0.)
Geometric  tensor(0.0011)
